In [1]:
import psycopg2
import pandas as pd

In [2]:
try:
    connection = psycopg2.connect(user = "mosiqi",
#                                   password = "pynative@#29",
                                  host = "127.0.0.1",
                                  database = "mimic")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

{'user': 'mosiqi', 'passfile': '/Users/mosiqi/.pgpass', 'dbname': 'mimic', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.4 on x86_64-apple-darwin19.5.0, compiled by Apple clang version 11.0.3 (clang-1103.0.32.62), 64-bit',) 

PostgreSQL connection is closed


In [3]:
connection = psycopg2.connect(user = "mosiqi",
#                                   password = "pynative@#29",
                              host = "127.0.0.1",
                              database = "mimic",
                              options=f'-c search_path=mimiciii') # set default search_path,no need to 'mimiciii.patients'

{'user': 'mosiqi', 'passfile': '/Users/mosiqi/.pgpass', 'dbname': 'mimic', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '-c search_path=mimiciii', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  (46520,) 



# Age

In [48]:
sql = """
WITH first_admission_time AS
(
  SELECT
      p.subject_id, p.dob, p.gender
      , MIN (a.admittime) AS first_admittime
      , MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
          AS first_admit_age
  FROM patients p
  INNER JOIN admissions a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender
  ORDER BY p.subject_id
)
SELECT
    subject_id, dob, gender
    , first_admittime, first_admit_age
    , CASE
        -- all ages > 89 in the database were replaced with 300
        WHEN first_admit_age > 89
            then '>89'
        WHEN first_admit_age >= 14
            THEN 'adult'
        WHEN first_admit_age <= 1
            THEN 'neonate'
        ELSE 'middle'
        END AS age_group
FROM first_admission_time
ORDER BY subject_id
"""
df = pd.read_sql(sql, connection)
df

,subject_id,dob,gender,first_admittime,first_admit_age,age_group
0,2,2138-07-17,M,2138-07-17 19:04:00,0.00,neonate
1,3,2025-04-11,M,2101-10-20 19:08:00,76.52,adult
2,4,2143-05-12,F,2191-03-16 00:28:00,47.84,adult
3,5,2103-02-02,M,2103-02-02 04:31:00,0.00,neonate
4,6,2109-06-21,F,2175-05-30 07:15:00,65.94,adult
...,...,...,...,...,...,...
46515,99985,2127-04-08,M,2181-01-27 02:47:00,53.81,adult
46516,99991,2137-04-07,M,2184-12-24 08:30:00,47.72,adult
46517,99992,2078-10-17,F,2144-07-25 18:03:00,65.77,adult
46518,99995,2058-05-29,F,2147-02-08 08:00:00,88.70,adult


## 计算APACHE II所需variables：
1. age, ok
2. temperature-rectal 摄氏度, ok
3. mean arterial pressure mmHG, ok
4. heart rate, ok
5. respiratory rate (non-ventilate or centilate), ok
6. Oxygenation, ok
7. Arterial PH, ok 
8. Serum sodium, ok
9. Serum potassium, ok
10. Serum creatinine, ok
11. hematocrit %, ok
12. white blood count, ok

13. Glasgow coma score, ok \
and other chronical disease

#### 均取most deranged physiology value （最大or最小，看哪个值更大取哪个）

## Basic Vital

In [65]:
sql = """
SELECT pvt.subject_id, pvt.hadm_id, pvt.icustay_id

-- Easier names
, min(case when VitalID = 1 then valuenum ELSE NULL END) AS heartrate_min
, max(case when VitalID = 1 then valuenum ELSE NULL END) AS heartrate_max
, min(case when VitalID = 2 then valuenum ELSE NULL END) AS tempc_min
, max(case when VitalID = 2 then valuenum ELSE NULL END) AS tempc_max
, min(case when VitalID = 3 then valuenum ELSE NULL END) AS meanbp_min
, max(case when VitalID = 3 then valuenum ELSE NULL END) AS meanbp_max
, min(case when VitalID = 4 then valuenum ELSE NULL END) AS resprate_min
, max(case when VitalID = 4 then valuenum ELSE NULL END) AS resprate_max

FROM  (
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 1 -- HeartRate
    when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 2 -- TempF, converted to degC in valuenum call
    when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 2 -- TempC
    when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 3 -- MeanBP
    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 4 -- RespRate
    
    else null end as vitalid
    , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from icustays ie
  left join chartevents ce
  on ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime and ie.intime + interval '24 hour'
  
    -- exclude rows marked as error
  and (ce.error IS NULL or ce.error = 0)
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"
  
    -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676, -- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678, -- "Temperature F"
  
    -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, -- Arterial BP Mean #2
  443, -- Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,-- Respiratory Rate
  615,-- Resp Rate (Total)
  220210,-- Respiratory Rate
  224690 -- Respiratory Rate (Total)
  
  )
) pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
"""
df = pd.read_sql(sql,connection)
df

,subject_id,hadm_id,icustay_id,heartrate_min,heartrate_max,tempc_min,tempc_max,meanbp_min,meanbp_max,resprate_min,resprate_max
0,2,163353,243653,140.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3,145834,211552,75.0,168.0,36.200001,37.599998,40.000000,259.000,5.0,24.0
2,4,185777,294638,74.0,111.0,36.333334,37.444445,69.000000,100.333,18.0,32.0
3,6,107064,228232,76.0,100.0,36.000002,37.333332,72.666702,100.000,8.0,20.0
4,7,118037,236754,115.0,121.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
59058,99985,176670,279638,74.0,121.0,37.111111,38.611111,56.000000,110.000,18.0,39.0
59059,99991,151118,226241,91.0,124.0,36.333333,37.611111,86.000000,175.000,16.0,38.0
59060,99992,197084,242052,60.0,83.0,36.111111,36.888889,48.000000,81.000,11.0,20.0
59061,99995,137810,229633,49.0,93.0,36.444444,37.611111,61.000000,103.000,11.0,27.0


## pao2/aado2 + ph
github里还计算了chartevent的po2合aado2，但其实计算apacheII最后用的还是labevent里面的

In [5]:
sql = """
with pvt as
( 
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
      , case
        when itemid = 50801 then 'AADO2'
        when itemid = 50816 then 'FIO2'
        when itemid = 50821 then 'PO2'
        when itemid = 50820 then 'PH'
        else null
        end as label
        , charttime
        , value
        -- add in some sanity checks on the values
        , case
          when itemid = 50816 and valuenum < 20 then null
          when itemid = 50816 and valuenum > 100 then null
          when itemid = 50821 and valuenum > 800 then null -- PO2
           -- conservative upper limit
        else valuenum
        end as valuenum

    FROM icustays ie
    left join labevents le
      on le.subject_id = ie.subject_id and le.hadm_id = ie.hadm_id
      and le.charttime between ie.intime + interval '6 hour' and ie.intime + interval '1 day'
      and le.ITEMID in
      -- blood gases
      (
        50801, 50816, 50821, 50820
      )
)
select pvt.SUBJECT_ID, pvt.HADM_ID, pvt.ICUSTAY_ID, pvt.CHARTTIME
, max(case when label = 'AADO2' then valuenum else null end) as aado2
, max(case when label = 'FIO2' then valuenum else null end) as fio2
, max(case when label = 'PO2' then valuenum else null end) as po2
, max(case when label = 'PH' then valuenum else null end) as ph
from pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME

"""
df = pd.read_sql(sql,connection)
df

,subject_id,hadm_id,icustay_id,charttime,aado2,fio2,po2,ph
0,2,163353,243653,NaT,NaN,NaN,NaN,NaN
1,3,145834,211552,2101-10-21 01:46:00,NaN,NaN,NaN,7.30
2,3,145834,211552,2101-10-21 03:09:00,NaN,60.0,181.0,7.33
3,3,145834,211552,2101-10-21 08:46:00,NaN,NaN,148.0,7.42
4,3,145834,211552,2101-10-21 08:51:00,NaN,NaN,27.0,7.37
...,...,...,...,...,...,...,...,...
121274,99991,151118,226241,2184-12-29 04:13:00,506.0,100.0,189.0,7.35
121275,99992,197084,242052,NaT,NaN,NaN,NaN,NaN
121276,99995,137810,229633,2147-02-08 20:36:00,NaN,NaN,146.0,7.40
121277,99995,137810,229633,2147-02-09 08:09:00,NaN,NaN,110.0,7.46


fio2<50 且 po2 not null: 7159 rows
fio2>= 50 且 aado2 not null：3314 rows
有效数据：1w

In [6]:
df1 = df[(df['fio2']>=50) & (df['aado2'].notnull())]
df1

,subject_id,hadm_id,icustay_id,charttime,aado2,fio2,po2,ph
36,21,109451,217847,2134-09-12 09:21:00,560.0,100.0,116.0,7.28
38,21,109451,217847,2134-09-12 13:52:00,618.0,100.0,33.0,7.11
40,21,109451,217847,2134-09-12 20:13:00,496.0,100.0,167.0,7.25
203,94,140037,213450,2176-09-03 07:30:00,347.0,100.0,328.0,7.49
293,111,155897,249202,2144-07-01 11:38:00,551.0,100.0,41.0,6.94
...,...,...,...,...,...,...,...,...
121155,99817,195557,263780,2166-11-20 19:49:00,467.0,100.0,242.0,6.98
121179,99836,101018,240303,2116-06-26 12:52:00,438.0,100.0,267.0,7.44
121213,99881,172327,214377,2174-01-21 09:57:00,333.0,80.0,168.0,7.17
121270,99985,176670,279638,2181-01-29 15:08:00,437.0,80.0,109.0,7.40


In [7]:
df2 = df[(df['fio2']<50) & (df['po2'].notnull())]
df2

,subject_id,hadm_id,icustay_id,charttime,aado2,fio2,po2,ph
6,3,145834,211552,2101-10-21 15:48:00,NaN,40.0,110.0,7.40
7,3,145834,211552,2101-10-21 15:52:00,NaN,40.0,32.0,7.36
8,3,145834,211552,2101-10-21 17:33:00,NaN,40.0,96.0,7.37
9,3,145834,211552,2101-10-21 17:37:00,NaN,40.0,34.0,7.30
64,31,128652,254478,2108-08-23 09:08:00,NaN,30.0,128.0,7.47
...,...,...,...,...,...,...,...,...
121112,99781,147562,200664,2133-08-02 10:09:00,NaN,40.0,94.0,7.46
121143,99810,196261,272409,2109-02-02 09:39:00,NaN,40.0,109.0,7.40
121216,99883,150755,276467,2131-12-25 08:34:00,NaN,40.0,83.0,7.38
121221,99897,162913,266801,2181-08-06 15:55:00,NaN,21.0,28.0,7.29


In [8]:
df_xyo = pd.concat([df1, df2])
df_xyo

,subject_id,hadm_id,icustay_id,charttime,aado2,fio2,po2,ph
36,21,109451,217847,2134-09-12 09:21:00,560.0,100.0,116.0,7.28
38,21,109451,217847,2134-09-12 13:52:00,618.0,100.0,33.0,7.11
40,21,109451,217847,2134-09-12 20:13:00,496.0,100.0,167.0,7.25
203,94,140037,213450,2176-09-03 07:30:00,347.0,100.0,328.0,7.49
293,111,155897,249202,2144-07-01 11:38:00,551.0,100.0,41.0,6.94
...,...,...,...,...,...,...,...,...
121112,99781,147562,200664,2133-08-02 10:09:00,NaN,40.0,94.0,7.46
121143,99810,196261,272409,2109-02-02 09:39:00,NaN,40.0,109.0,7.40
121216,99883,150755,276467,2131-12-25 08:34:00,NaN,40.0,83.0,7.38
121221,99897,162913,266801,2181-08-06 15:55:00,NaN,21.0,28.0,7.29


### pao2/aado2 score assignment
can be done by pandas

In [ ]:
    case
      when pao2 is null and aado2 is null
        then null
      when pao2 is not null then
        case
          when pao2 < 50 then 15
          when pao2 < 70 then 5
          when pao2 < 80 then 2
        else 0 end
      when aado2 is not null then
        case
          when aado2 <  100 then 0
          when aado2 <  250 then 7
          when aado2 <  350 then 9
          when aado2 <  500 then 11
          when aado2 >= 500 then 14
        else 0 end
      end as pao2_aado2_score

# sodium, potassium, creatinine, hematocrit, white blood count

In [9]:
sql = """
SELECT
  pvt.subject_id, pvt.hadm_id, pvt.icustay_id

  , min(CASE WHEN label = 'CREATININE' THEN valuenum ELSE NULL END) AS creatinine_min
  , max(CASE WHEN label = 'CREATININE' THEN valuenum ELSE NULL END) AS creatinine_max
  , min(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE NULL END) AS hematocrit_min
  , max(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE NULL END) AS hematocrit_max
  , min(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE NULL END) AS potassium_min
  , max(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE NULL END) AS potassium_max
  , min(CASE WHEN label = 'SODIUM' THEN valuenum ELSE NULL END) AS sodium_min
  , max(CASE WHEN label = 'SODIUM' THEN valuenum ELSE NULL END) AS sodium_max
  , min(CASE WHEN label = 'WBC' THEN valuenum ELSE NULL END) AS wbc_min
  , max(CASE WHEN label = 'WBC' THEN valuenum ELSE NULL END) AS wbc_max

FROM
( 
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id
  , CASE
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END as label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE

      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
    ELSE le.valuenum
    END as valuenum

  FROM icustays ie

  LEFT JOIN labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime between ie.intime + interval '6 hour' and ie.intime + interval '1 day'
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id
ORDER BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
"""
df3 = pd.read_sql(sql,connection)
df3

,subject_id,hadm_id,icustay_id,creatinine_min,creatinine_max,hematocrit_min,hematocrit_max,potassium_min,potassium_max,sodium_min,sodium_max,wbc_min,wbc_max
0,2,163353,243653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,145834,211552,2.5,2.5,27.0,38.0,3.9,4.9,138.0,139.0,18.8,24.4
2,4,185777,294638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,178980,214757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,107064,228232,NaN,NaN,25.4,34.4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,99985,176670,279638,1.0,1.0,28.9,28.9,3.9,3.9,134.0,134.0,12.3,12.3
61528,99991,151118,226241,0.9,1.0,29.8,29.8,3.4,4.2,143.0,146.0,7.3,7.3
61529,99992,197084,242052,1.2,1.2,28.6,29.3,4.9,4.9,136.0,136.0,17.1,17.1
61530,99995,137810,229633,0.5,0.5,28.7,28.7,3.5,4.1,135.0,136.0,7.3,7.3


In [11]:
df3.isna().sum()

subject_id            0
hadm_id               0
icustay_id            0
creatinine_min    16647
creatinine_max    16647
hematocrit_min    15458
hematocrit_max    15458
potassium_min     13615
potassium_max     13615
sodium_min        14660
sodium_max        14660
wbc_min           18350
wbc_max           18350
dtype: int64

## Glasgow coma score 
1. eye
2. verbal
3. motor

In [20]:
sql = """
with base as
(
  SELECT pvt.ICUSTAY_ID
  , pvt.charttime

  -- Easier names - note we coalesced Metavision and CareVue IDs below
  , max(case when pvt.itemid = 454 then pvt.valuenum else null end) as GCSMotor
  , max(case when pvt.itemid = 723 then pvt.valuenum else null end) as GCSVerbal
  , max(case when pvt.itemid = 184 then pvt.valuenum else null end) as GCSEyes

  -- If verbal was set to 0 in the below select, then this is an intubated patient
  , case
      when max(case when pvt.itemid = 723 then pvt.valuenum else null end) = 0
    then 1
    else 0
    end as EndoTrachFlag

  , ROW_NUMBER ()
          OVER (PARTITION BY pvt.ICUSTAY_ID ORDER BY pvt.charttime ASC) as rn

  FROM  
  (
  select l.ICUSTAY_ID
  -- merge the ITEMIDs so that the pivot applies to both metavision/carevue data
  , case
      when l.ITEMID in (723,223900) then 723
      when l.ITEMID in (454,223901) then 454
      when l.ITEMID in (184,220739) then 184
      else l.ITEMID end
    as ITEMID

  -- convert the data into a number, reserving a value of 0 for ET/Trach
  , case
      -- endotrach/vent is assigned a value of 0, later parsed specially
      when l.ITEMID = 723 and l.VALUE = '1.0 ET/Trach' then 0 -- carevue
      when l.ITEMID = 223900 and l.VALUE = 'No Response-ETT' then 0 -- metavision

      else VALUENUM
      end
    as VALUENUM
  , l.CHARTTIME
  FROM chartevents l

  -- get intime for charttime subselection
  inner join icustays b
    on l.icustay_id = b.icustay_id

  -- Isolate the desired GCS variables
  where l.ITEMID in
  (
    -- 198 -- GCS
    -- GCS components, CareVue
    184, 454, 723
    -- GCS components, Metavision
    , 223900, 223901, 220739
  )
  -- Only get data for the first 24 hours
  and l.charttime between b.intime and b.intime + INTERVAL '1 day'
  -- exclude rows marked as error
  AND (l.error IS NULL OR l.error = 0)
  ) pvt
  group by pvt.ICUSTAY_ID, pvt.charttime
)
, gcs as (
  select b.*
  , b2.GCSVerbal as GCSVerbalPrev
  , b2.GCSMotor as GCSMotorPrev
  , b2.GCSEyes as GCSEyesPrev
  -- Calculate GCS, factoring in special case when they are intubated and prev vals
  -- note that the coalesce are used to implement the following if:
  --  if current value exists, use it
  --  if previous value exists, use it
  --  otherwise, default to normal
  , case
      -- replace GCS during sedation with 15
      when b.GCSVerbal = 0
        then 15
      when b.GCSVerbal is null and b2.GCSVerbal = 0
        then 15
      -- if previously they were intub, but they aren't now, do not use previous GCS values
      when b2.GCSVerbal = 0
        then
            coalesce(b.GCSMotor,6)
          + coalesce(b.GCSVerbal,5)
          + coalesce(b.GCSEyes,4)
      -- otherwise, add up score normally, imputing previous value if none available at current time
      else
            coalesce(b.GCSMotor,coalesce(b2.GCSMotor,6))
          + coalesce(b.GCSVerbal,coalesce(b2.GCSVerbal,5))
          + coalesce(b.GCSEyes,coalesce(b2.GCSEyes,4))
      end as GCS

  from base b
  -- join to itself within 6 hours to get previous value
  left join base b2
    on b.ICUSTAY_ID = b2.ICUSTAY_ID 
    and b.rn = b2.rn+1 
    and b2.charttime > b.charttime - INTERVAL '6 hour'
)
, gcs_final as (
  select gcs.*
  -- This sorts the data by GCS, so rn=1 is the the lowest GCS values to keep
  , ROW_NUMBER ()
          OVER (PARTITION BY gcs.ICUSTAY_ID
                ORDER BY gcs.GCS
               ) as IsMinGCS
  from gcs
)

select ie.subject_id, ie.hadm_id, ie.icustay_id
-- The minimum GCS is determined by the above row partition, we only join if IsMinGCS=1
, GCS as mingcs
, coalesce(GCSMotor,GCSMotorPrev) as gcsmotor
, coalesce(GCSVerbal,GCSVerbalPrev) as gcsverbal
, coalesce(GCSEyes,GCSEyesPrev) as gcseyes
, EndoTrachFlag as endotrachflag

-- subselect down to the cohort of eligible patients
FROM icustays ie
left join gcs_final gs
  on ie.icustay_id = gs.icustay_id and gs.IsMinGCS = 1
ORDER BY ie.icustay_id;
"""
df4 = pd.read_sql(sql,connection)
df4

,subject_id,hadm_id,icustay_id,mingcs,gcsmotor,gcsverbal,gcseyes,endotrachflag
0,55973,152234,200001,14.0,6.0,5.0,3.0,0.0
1,27513,163557,200003,15.0,4.0,0.0,2.0,1.0
2,10950,189514,200006,15.0,6.0,5.0,4.0,0.0
3,20707,129310,200007,15.0,6.0,5.0,4.0,0.0
4,29904,129607,200009,15.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...
61527,13620,169431,299993,15.0,6.0,5.0,4.0,0.0
61528,10718,177406,299994,15.0,6.0,5.0,4.0,0.0
61529,28775,134959,299995,15.0,4.0,0.0,2.0,1.0
61530,69587,158288,299998,15.0,5.0,0.0,2.0,1.0
